In [1]:
!pip install pandas;
!pip install scipy;
import pandas as pd;
from pathlib import Path;
import numpy as np;


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
data_dir = Path("data")
outbreak = pd.read_csv(data_dir / "OutbreaksData.csv")
outbreak.head()

,Unnamed: 0,Country,iso2,iso3,Year,icd10n,icd103n,icd104n,icd10c,icd103c,icd104c,icd11c1,icd11c2,icd11c3,icd11l1,icd11l2,icd11l3,Disease,DONs,Definition
0,1,Afghanistan,AF,AFG,2000,Arthropod-borne viral fevers and viral haemorr...,Unspecified viral haemorrhagic fever,Unspecified viral haemorrhagic fever,A92-A99,A99,A990,1,D86,1D86,Certain infectious or parasitic diseases,Certain other viral diseases,"Viral haemorrhagic fever, not elsewhere classi...",Unspecified viral haemorrhagic fever,"DON0404, DON0399",NaN
1,2,Afghanistan,AF,AFG,2002,Intestinal infectious diseases,Other gastroenteritis and colitis of infectiou...,Other and unspecified gastroenteritis and coli...,A00-A09,A09,A090,1,A40,1A40,Certain infectious or parasitic diseases,Gastroenteritis or colitis of infectious origin,Gastroenteritis or colitis without specificati...,Infectious gastroenteritis or colitis without ...,DON0607,NaN
2,3,Afghanistan,AF,AFG,1998,Intestinal infectious diseases,Cholera,Classical cholera,A00-A09,A00,A000,1,A00,1A00,Certain infectious or parasitic diseases,Gastroenteritis or colitis of infectious origin,Bacterial intestinal infections,Cholera,"DON0253, DON0249, DON0248",Intestinal infection due to Vibrio cholerae
3,4,Afghanistan,AF,AFG,1999,Codes for special purposes,Provisional assignment of new diseases of unce...,Severe acute respiratory syndrome [SARS],U00-U49,U04,U049,1,D65,1D65,Certain infectious or parasitic diseases,Certain zoonotic viral diseases,Severe acute respiratory syndrome,Severe acute respiratory syndrome,DON0294,"A disease of the respiratory system, caused by..."
4,5,Afghanistan,AF,AFG,2000,Intestinal infectious diseases,Cholera,Classical cholera,A00-A09,A00,A000,1,A00,1A00,Certain infectious or parasitic diseases,Gastroenteritis or colitis of infectious origin,Bacterial intestinal infections,Cholera,DON0413,Intestinal infection due to Vibrio cholerae


In [3]:
zoonotic = [
    'Unspecified viral haemorrhagic fever',
    'Severe acute respiratory syndrome [SARS]',
    'Influenza due to identified zoonotic or pandemic influenza virus',
    'Dengue, unspecified',
    'Leishmaniasis, unspecified',
    'West Nile virus infection',
    'Plague, unspecified',
    'Middle East respiratory syndrome coronavirus [MERS-CoV]',
    'Yellow fever, unspecified',
    'Hantavirus (cardio-)pulmonary syndrome',
    'Chikungunya virus disease',
    'Listeriosis, unspecified',
    'Legionnaires disease',
    'Acute hepatitis A',
    'Zika virus disease',
    'Lassa fever',
    'Acute hepatitis E',
    'Monkeypox',
    'Leptospirosis, unspecified',
    'Rift Valley fever',
    'Ebola virus disease',
    'Marburg virus disease',
    'Anthrax, unspecified',
    'Rabies, unspecified',
    'Salmonella infection, unspecified',
    'Tularaemia, unspecified',
    'Venezuelan equine fever',
    'Coccidioidomycosis, unspecified',
    'Haemorrhagic fever with renal syndrome',
    'St Louis encephalitis',
    'Bubonic plague'
]
# We are not including covid, just adding an outbreak in China since this was the animal to human part


In [4]:
filtered_df = outbreak[outbreak['icd103n'].isin(zoonotic)]

In [5]:
filtered_df['Year'].unique()

array([2000, 2009, 1998, 2014, 2004, 2005, 2010, 2016, 2017, 2006, 2003,
       2008, 2012, 2011, 1996, 2002, 2020, 2021, 2013, 2007, 2018, 2015,
       2001, 1999, 1997, 2019, 2022], dtype=int64)

In [6]:
final_outbreak = filtered_df[['Year', 'Country', 'icd103n']]
final_outbreak._append({"Year": 2020, "Country": "China", 'icd103n': 'COVID-19, virus identified'}, ignore_index=True)

,Year,Country,icd103n
0,2000,Afghanistan,Unspecified viral haemorrhagic fever
1,2009,Afghanistan,Influenza due to identified zoonotic or pandem...
2,1998,Afghanistan,Unspecified viral haemorrhagic fever
3,2009,Albania,Influenza due to identified zoonotic or pandem...
4,2009,Algeria,Influenza due to identified zoonotic or pandem...
...,...,...,...
546,2014,Yemen,Severe acute respiratory syndrome [SARS]
547,2009,Zambia,Influenza due to identified zoonotic or pandem...
548,1999,Zimbabwe,Unspecified viral haemorrhagic fever
549,2009,Zimbabwe,Influenza due to identified zoonotic or pandem...


In [7]:
country_data_raw = pd.read_csv(data_dir / "country_data" / "country_data.csv")
country_data_raw.columns

Index(['Series Name', 'Series Code', 'Country Name', 'Country Code',
       '1960 [YR1960]', '1961 [YR1961]', '1962 [YR1962]', '1963 [YR1963]',
       '1964 [YR1964]', '1965 [YR1965]', '1966 [YR1966]', '1967 [YR1967]',
       '1968 [YR1968]', '1969 [YR1969]', '1970 [YR1970]', '1971 [YR1971]',
       '1972 [YR1972]', '1973 [YR1973]', '1974 [YR1974]', '1975 [YR1975]',
       '1976 [YR1976]', '1977 [YR1977]', '1978 [YR1978]', '1979 [YR1979]',
       '1980 [YR1980]', '1981 [YR1981]', '1982 [YR1982]', '1983 [YR1983]',
       '1984 [YR1984]', '1985 [YR1985]', '1986 [YR1986]', '1987 [YR1987]',
       '1988 [YR1988]', '1989 [YR1989]', '1990 [YR1990]', '1991 [YR1991]',
       '1992 [YR1992]', '1993 [YR1993]', '1994 [YR1994]', '1995 [YR1995]',
       '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]', '1999 [YR1999]',
       '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]', '2003 [YR2003]',
       '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]', '2007 [YR2007]',
       '2008 [YR2008]', '2009 [

In [8]:
filtered_columns = ['Series Name', 'Country Name', '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]', '1999 [YR1999]',
       '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]', '2003 [YR2003]',
       '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]', '2007 [YR2007]',
       '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]', '2011 [YR2011]',
       '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]', '2015 [YR2015]',
       '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]', '2019 [YR2019]',
       '2020 [YR2020]', '2021 [YR2021]', '2022 [YR2022]']
country_data = country_data_raw[filtered_columns].iloc[:-5] # Getting rid of data description at the bottom

In [9]:
# Making custom interpolation function since np's is being weird
def convert_values(series):
    return [np.nan if num == '..' else pd.to_numeric(num) for num in series]

In [10]:
# converting strings to nan
dropped_cols = ['Series Name', 'Country Name'] 
df_subset = country_data.drop(columns=dropped_cols)
df_filled = df_subset.apply(convert_values)
print("Converted non numerics to nan")
country_final = pd.concat([country_data[dropped_cols], df_filled], axis=1)
country_final.head()

Converted non numerics to nan


,Series Name,Country Name,1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,"Completeness of birth registration, rural (%)",Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3,...,NaN,NaN,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Completeness of birth registration, rural (%)",Africa Eastern and Southern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Completeness of birth registration, rural (%)",Africa Western and Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Completeness of birth registration, rural (%)",Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,98.4,NaN,NaN,NaN,NaN
4,"Completeness of birth registration, rural (%)",Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,99.5,NaN,NaN,NaN,NaN,NaN,99.6,NaN,NaN,NaN


In [11]:
#
final_outbreak_ml = final_outbreak[['Year', 'Country']]
final_outbreak_ml = final_outbreak_ml.groupby(['Country', 'Year']).size().unstack(fill_value=0)
final_outbreak_ml.head()                                     

Year,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Albania,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Algeria,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
American Samoa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Andorra,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Normalize the two tables so that the years and countries match up

In [12]:
# Are there any mismatches between the two?
countries1 = np.array([str(country) for country in country_final['Country Name'].unique()])
countries2 = np.array(final_outbreak_ml.index)

In [13]:
np.setdiff1d(countries1, countries2)

array(['Africa Eastern and Southern', 'Africa Western and Central',
       'Arab World', 'Bahamas, The', 'Belarus', 'Benin', 'Bolivia',
       'British Virgin Islands', 'Burkina Faso', 'Caribbean small states',
       'Central Europe and the Baltics', 'Channel Islands', 'Comoros',
       'Congo, Dem. Rep.', 'Congo, Rep.', "Cote d'Ivoire", 'Curacao',
       'Early-demographic dividend', 'East Asia & Pacific',
       'East Asia & Pacific (IDA & IBRD countries)',
       'East Asia & Pacific (excluding high income)', 'Egypt, Arab Rep.',
       'Equatorial Guinea', 'Eritrea', 'Euro area',
       'Europe & Central Asia',
       'Europe & Central Asia (IDA & IBRD countries)',
       'Europe & Central Asia (excluding high income)', 'European Union',
       'Faroe Islands', 'Fragile and conflict affected situations',
       'Gambia, The', 'Gibraltar', 'Greenland', 'Guinea-Bissau',
       'Heavily indebted poor countries (HIPC)', 'High income',
       'Hong Kong SAR, China', 'IBRD only', 'IDA & 

In [14]:
np.setdiff1d(countries2, countries1)


array(['Bahamas', 'Bolivia (Plurinational State of)',
       'Bonaire Sint Eustatius and Saba', 'Congo',
       'Congo Democratic Republic of the', 'Cook Islands', 'Curaçao',
       "Côte d'Ivoire", 'Egypt', 'Falkland Islands (Malvinas)',
       'French Guiana', 'Guadeloupe', 'Guernsey', 'Hong Kong',
       'Iran (Islamic Republic of)', 'Jersey',
       "Korea (Democratic People's Republic of)", 'Korea Republic of',
       "Lao People's Democratic Republic", 'Martinique',
       'Micronesia (Federated States of)', 'Moldova Republic of',
       'Palestine State of', 'Réunion', 'Saint Kitts and Nevis',
       'Saint Lucia', 'Saint Martin (French part)',
       'Saint Vincent and the Grenadines', 'Slovakia',
       'Tanzania United Republic of', 'Turkey',
       'United Kingdom of Great Britain and Northern Ireland',
       'United States of America', 'Venezuela (Bolivarian Republic of)',
       'Viet Nam', 'Virgin Islands (British)', 'Wallis and Futuna',
       'Yemen'], dtype=object)

In [15]:
def update_name(name):
    lookup = {
        "Bolivia (Plurinational State of)": "Bolivia",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "Turkey": "Turkiye",
        "United States of America": "United States",
        "Viet Nam": "Vietnam",
        "Yemen": "Yemen, Rep.",
        "Egypt": "Egypt, Arab Rep.",
        "Hong Kong": "Hong Kong SAR, China",
        "Iran (Islamic Republic of)": "Iran, Islamic Rep.",
        "Venezuela (Bolivarian Republic of)": "Venezuela, RB",
        "Congo Democratic Republic of the": "Congo, Dem. Rep.",
        "Korea Republic of": "Korea, Rep."
    }
    if name in lookup:
        return lookup[name]
    else:
        return name

final_outbreak_ml.index = pd.Series(final_outbreak_ml.index).apply(update_name)

In [16]:
# Subset world data for only countries with outbreaks
country_norm = country_final[country_final['Country Name'].isin(final_outbreak_ml.index)]
outbreak_norm = final_outbreak_ml.loc[country_norm['Country Name'].unique()]

## Merging the two tables together
Now that the tables have the same countries, we can try to merge them together

In [17]:
country_long = country_norm.melt(id_vars=["Series Name", "Country Name"], var_name="year", value_name="value")
def extract_year(year_str):
    return year_str.split(" ")[0]
country_long['year'] = country_long['year'].apply(extract_year)
country_long.head()

,Series Name,Country Name,year,value
0,"Completeness of birth registration, rural (%)",Afghanistan,1996,NaN
1,"Completeness of birth registration, rural (%)",Albania,1996,NaN
2,"Completeness of birth registration, rural (%)",Algeria,1996,NaN
3,"Completeness of birth registration, rural (%)",American Samoa,1996,NaN
4,"Completeness of birth registration, rural (%)",Andorra,1996,NaN


In [18]:
outbreak_long = outbreak_norm.reset_index().rename(columns={"index":"Country"})
outbreak_long.head()

Year,Country,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Albania,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Algeria,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,American Samoa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Andorra,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Lastly, make sure the column names match exactly
outbreak_long = outbreak_long.reset_index(drop=True)
outbreak_long = outbreak_long.melt(id_vars="Country", var_name="Year", value_name="Outbreaks")
outbreak_long["Value"] = outbreak_long["Outbreaks"]
outbreak_long["Statistic"] = ["Outbreaks" for num in outbreak_long["Value"]]
outbreak_long.drop(columns=["Outbreaks"], inplace=True)
outbreak_long.head()

,Country,Year,Value,Statistic
0,Afghanistan,1996,0,Outbreaks
1,Albania,1996,0,Outbreaks
2,Algeria,1996,0,Outbreaks
3,American Samoa,1996,0,Outbreaks
4,Andorra,1996,0,Outbreaks


In [20]:
#Make sure they have the same column names
country_long.columns = ["Statistic", "Country", "Year", "Value"]

In [22]:
training_df = pd.concat([outbreak_long, country_long])
training_df.head()